In [1527]:
# save csv as data frame 

import pandas as pd
import numpy as np
import re


# Clean Up/ Data Wrangling

# Please note that within this dataset, Nan values will be treated as 0. 

In [1528]:
demographics_csv_files = ["ACSDP1Y2010.DP05-Data.csv",
"ACSDP5Y2021.DP05-Data.csv",
"ACSDP1Y2011.DP05-Data.csv",
"ACSDP1Y2012.DP05-Data.csv",
"ACSDP1Y2013.DP05-Data.csv",
"ACSDP1Y2014.DP05-Data.csv",
"ACSDP1Y2015.DP05-Data.csv",
"ACSDP1Y2016.DP05-Data.csv",
"ACSDP1Y2017.DP05-Data.csv",
"ACSDP5Y2012.DP05-Data.csv",
"ACSDP5Y2011.DP05-Data.csv",
"ACSDP5Y2010.DP05-Data.csv",
"ACSDP1Y2021.DP05-Data.csv",
"ACSDP1Y2019.DP05-Data.csv",
"ACSDP1Y2018.DP05-Data.csv"]

In [1529]:
def clean_census_df(_census_df, year):
    # remove all the weird values for estimates
    census_df = _census_df.replace({'**': np.nan, 
                           '-': np.nan,
                           '***': np.nan, 
                           '*****': np.nan, 
                           '(X)': np.nan, 
                           'N/A': np.nan,
                           'null': np.nan, 
                           'median-': np.nan, 
                           'median+': np.nan,
                          '-666666666': np.nan, 
                          'N': np.nan,
                          '-888888888': np.nan,
                          '-Varies': np.nan,
                          '-222222222': np.nan,
                          '-333333333': np.nan,
                          '*': np.nan,})
    census_df = census_df.dropna(how='all', axis='columns').dropna(how='all')
    
    if census_df.empty:
        return None
    
    print(year)
    # Re-index using the "Labels" Columns
    # Reference : https://api.census.gov/data/2021/acs/acs5/profile/groups/DP05.html 
    
    census_df.columns = pd.MultiIndex.from_frame(pd.MultiIndex.from_tuples(census_df.columns.str.split('!!').tolist()).to_frame().fillna(""))

    census_df = census_df.T.reset_index()
    
#     census_df.columns = pd.MultiIndex.from_frame(pd.MultiIndex.from_tuples(census_df.columns.str.split('!!').tolist()).to_frame().fillna(""))
#     census_df = census_df.T.reset_index()
    
    # Get relevant columns 
    census_df = census_df[census_df[0].str.contains('Zip Code') | census_df[0].str.contains('Estimate')]
    
    # Update the index and drop unnecessary columns
    multi_index_columns = census_df.apply(lambda x: x.str.isnumeric()==False).all()[census_df.apply(lambda x: x.str.isnumeric()==False).all()].index
    
    census_df.index = pd.MultiIndex.from_frame(census_df[multi_index_columns])

    census_df = census_df.drop(columns=census_df.index.names)    
    
    # Fill straggling NaNs
    
    census_df = census_df.fillna(0)
    
    # Add year
    census_df.loc[('Year'),:] = year
    
    census_df = census_df.T
    census_df = census_df.set_index(['Year','Zip Code'])
    
    census_df.columns = census_df.columns.droplevel(0)

    
    return census_df
    


In [1530]:
valid_census_dfs = []
for file in demographics_csv_files:
    year = re.search("([0-9]{4})", file)[0]
    file_path = './data/raw/' + file
    df = pd.read_csv(file_path, header=1, dtype=str)
    temp_df = df
    temp_df = temp_df.loc[temp_df['Geographic Area Name'].str.contains('ZCTA')]
    temp_df['Zip Code'] = temp_df['Geographic Area Name'][temp_df['Geographic Area Name'].str.contains('ZCTA')].str.split(" ").str[1]
    
    df = clean_census_df(temp_df, year)
    
    if df is None: 
        continue
    else: 
#         print(file)
#         df = clean_census_df(temp_df, year)
        valid_census_dfs.append(df)
        
len(valid_census_dfs)



/var/folders/ht/q4d_vppn3x5d3_p7bs499mdh0000gn/T/ipykernel_74377/2348089967.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['Zip Code'] = temp_df['Geographic Area Name'][temp_df['Geographic Area Name'].str.contains('ZCTA')].str.split(" ").str[1]


2021


/var/folders/ht/q4d_vppn3x5d3_p7bs499mdh0000gn/T/ipykernel_74377/2348089967.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['Zip Code'] = temp_df['Geographic Area Name'][temp_df['Geographic Area Name'].str.contains('ZCTA')].str.split(" ").str[1]


2012


/var/folders/ht/q4d_vppn3x5d3_p7bs499mdh0000gn/T/ipykernel_74377/2348089967.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['Zip Code'] = temp_df['Geographic Area Name'][temp_df['Geographic Area Name'].str.contains('ZCTA')].str.split(" ").str[1]


2011


3

In [1531]:
valid_census_dfs[0].columns = valid_census_dfs[0].columns.droplevel(0)

In [1532]:
df = pd.concat(valid_census_dfs, axis=1).fillna(0)
df.columns = pd.MultiIndex.from_tuples(df.columns.tolist())
df=df.reset_index()

/var/folders/ht/q4d_vppn3x5d3_p7bs499mdh0000gn/T/ipykernel_74377/772371714.py:3: PerformanceWarning: indexing past lexsort depth may impact performance.
  df=df.reset_index()
/var/folders/ht/q4d_vppn3x5d3_p7bs499mdh0000gn/T/ipykernel_74377/772371714.py:3: PerformanceWarning: indexing past lexsort depth may impact performance.
  df=df.reset_index()


In [1533]:
# make df to csv 
df.to_csv('./data/clean/demographics.csv')